In [49]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [50]:
rawHisTransDF=pd.read_csv('Historical-transaction-data.csv')
rawStoreInfDF=pd.read_csv('Store-info.csv')
rawTestDF=pd.read_csv('Testing-data.csv')
rawHisTransDF.dropna(subset=['item_description','invoice_id'], inplace=True)
rawHisTransDF=rawHisTransDF.drop_duplicates()
# convert the date string column to datetime
rawHisTransDF['transaction_date'] = pd.to_datetime(rawHisTransDF['transaction_date'], format='%Y/%m/%d').dt.date
rawHisTransDF.dropna(subset=['item_description','invoice_id'], inplace=True)
rawHisTransDF=rawHisTransDF.drop_duplicates()

In [51]:
rawHisTransDF

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold
0,ORANGE BARLEY 1.5L,2021-12-11,147.0,BGXA,SHOP008,220,2
1,GINGER BEER 1.5L,2021-10-17,371.0,IA25,SHOP112,220,2
2,TONIC PET 500ML,2021-12-13,484.0,VN7V,SHOP008,160,2
3,CREAM SODA 1L,2021-12-13,484.0,VN7V,SHOP008,150,2
4,STRAWBERRY MILK 180ML,2021-10-23,1310.0,7S00,SHOP112,210,5
...,...,...,...,...,...,...,...
473820,FIT O ORANGE 200ML,2021-11-14,8014206.0,ZEBZ,SHOP003,60,2
473821,BOTTLED DRINKING WATER 500ML,2021-10-16,8304754.0,AFR6,SHOP127,35,2
473822,GINGER BEER 1.5L,2021-10-16,8304807.0,J2A6,SHOP127,220,1
473823,BOTTLED DRINKING WATER 500ML,2021-11-07,8313570.0,1S0U,SHOP127,35,1


In [52]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
rawHisTransDF['item_description'] = le.fit_transform(rawHisTransDF['item_description'])
rawHisTransDF['customer_id'] = le.fit_transform(rawHisTransDF['customer_id'])
rawHisTransDF['shop_id'] = rawHisTransDF['shop_id'].str.replace(r'^SHOP', '').astype(int)
rawStoreInfDF['shop_id'] = rawStoreInfDF['shop_id'].str.replace(r'^SHOP', '').astype(int)
rawStoreInfDF['shop_profile'] = rawStoreInfDF['shop_profile'].replace({'High': 3, 'Moderate': 2, 'Low': 1})
rawStoreInfDF['shop_profile'] = rawStoreInfDF['shop_profile'].fillna(0.0).astype(int)
rawHisTransDF['invoice_id'] = rawHisTransDF['invoice_id'].astype(int)

C:\Users\DELL\AppData\Local\Temp\ipykernel_8244\3367044723.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  rawHisTransDF['shop_id'] = rawHisTransDF['shop_id'].str.replace(r'^SHOP', '').astype(int)
C:\Users\DELL\AppData\Local\Temp\ipykernel_8244\3367044723.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  rawStoreInfDF['shop_id'] = rawStoreInfDF['shop_id'].str.replace(r'^SHOP', '').astype(int)


In [53]:
rawHisTransDF

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold
0,27,2021-12-11,147,57272,8,220,2
1,14,2021-10-17,371,91334,112,220,2
2,35,2021-12-13,484,158179,8,160,2
3,4,2021-12-13,484,158179,8,150,2
4,34,2021-10-23,1310,38862,112,210,5
...,...,...,...,...,...,...,...
473820,13,2021-11-14,8014206,176899,3,60,2
473821,1,2021-10-16,8304754,52159,127,35,2
473822,14,2021-10-16,8304807,95280,127,220,1
473823,1,2021-11-07,8313570,8748,127,35,1


In [54]:
rawHisTransDF['payment']=rawHisTransDF['item_price']*rawHisTransDF['quantity_sold']

In [55]:
# group the dataframe by the 'group' column and get the size of each group
transactions_by_shop = rawHisTransDF.groupby('shop_id').size().reset_index()

# rename columns of the new dataframe
transactions_by_shop.columns = ['shop_id', 'num_of_transactions']

In [56]:
transactions_by_shop

,shop_id,num_of_transactions
0,1,1719
1,2,4115
2,3,3199
3,4,3000
4,5,3421
...,...,...
119,123,1714
120,124,3743
121,125,1449
122,126,2759


In [57]:
rawHisTransDF

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,payment
0,27,2021-12-11,147,57272,8,220,2,440
1,14,2021-10-17,371,91334,112,220,2,440
2,35,2021-12-13,484,158179,8,160,2,320
3,4,2021-12-13,484,158179,8,150,2,300
4,34,2021-10-23,1310,38862,112,210,5,1050
...,...,...,...,...,...,...,...,...
473820,13,2021-11-14,8014206,176899,3,60,2,120
473821,1,2021-10-16,8304754,52159,127,35,2,70
473822,14,2021-10-16,8304807,95280,127,220,1,220
473823,1,2021-11-07,8313570,8748,127,35,1,35


In [58]:
result = rawHisTransDF.groupby(['shop_id'])['payment'].sum()

In [59]:
# merging two dataframes on 'key' column
merged_df = pd.merge(rawStoreInfDF, result, on='shop_id')
merged_df = pd.merge(transactions_by_shop,result,on='shop_id')
merged_df

,shop_id,num_of_transactions,payment
0,1,1719,1002935
1,2,4115,2645315
2,3,3199,1667950
3,4,3000,1420415
4,5,3421,1610210
...,...,...,...
119,123,1714,825390
120,124,3743,2548215
121,125,1449,966570
122,126,2759,1623175


In [60]:
rawHisTransDF

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,payment
0,27,2021-12-11,147,57272,8,220,2,440
1,14,2021-10-17,371,91334,112,220,2,440
2,35,2021-12-13,484,158179,8,160,2,320
3,4,2021-12-13,484,158179,8,150,2,300
4,34,2021-10-23,1310,38862,112,210,5,1050
...,...,...,...,...,...,...,...,...
473820,13,2021-11-14,8014206,176899,3,60,2,120
473821,1,2021-10-16,8304754,52159,127,35,2,70
473822,14,2021-10-16,8304807,95280,127,220,1,220
473823,1,2021-11-07,8313570,8748,127,35,1,35


In [ ]:
temp1=rawHisTransDF['shop']

In [61]:
unique_values = rawHisTransDF['item_description'].unique()
print(unique_values)

[27 14 35  4 34  3 36 11 33 10 19 22 12 24 20 15  9  8  5 13  6  1 32 17
 31 26 30  0 25 18 28  7 21 29 16  2 23]


In [62]:
# group the dataframe by the 'group' column and get the size of each group
temp = rawHisTransDF.groupby(['shop_id','item_description']).size().reset_index()

# rename columns of the new dataframe
temp.columns = ['shop_id', 'item_description','times']

In [63]:
temp.to_csv('final1.csv')

In [70]:
# pivot the dataframe to make item_description values as columns
pivoted_df = temp.pivot(index='shop_id', columns='item_description', values='times')

# fill NaN values with 0
pivoted_df.fillna(0, inplace=True)

# resetting the index
pivoted_df = pivoted_df.reset_index()

In [71]:
pivoted_df

item_description,shop_id,0,1,2,3,4,5,6,7,8,...,27,28,29,30,31,32,33,34,35,36
0,1,28.0,64.0,0.0,194.0,16.0,48.0,55.0,11.0,24.0,...,117.0,32.0,10.0,16.0,58.0,71.0,38.0,15.0,54.0,13.0
1,2,66.0,121.0,0.0,503.0,41.0,236.0,92.0,46.0,40.0,...,211.0,104.0,13.0,45.0,129.0,190.0,207.0,24.0,150.0,28.0
2,3,73.0,164.0,0.0,325.0,48.0,156.0,129.0,23.0,62.0,...,163.0,66.0,7.0,28.0,107.0,140.0,121.0,36.0,134.0,21.0
3,4,44.0,79.0,0.0,279.0,33.0,105.0,55.0,24.0,30.0,...,164.0,62.0,11.0,34.0,127.0,155.0,103.0,38.0,81.0,33.0
4,5,62.0,96.0,0.0,337.0,33.0,143.0,97.0,35.0,64.0,...,143.0,76.0,8.0,35.0,126.0,112.0,122.0,49.0,84.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,123,41.0,51.0,0.0,173.0,18.0,76.0,59.0,16.0,38.0,...,87.0,37.0,8.0,19.0,63.0,60.0,52.0,42.0,42.0,22.0
120,124,87.0,172.0,0.0,254.0,34.0,132.0,72.0,74.0,92.0,...,155.0,69.0,12.0,31.0,128.0,179.0,169.0,103.0,139.0,49.0
121,125,41.0,92.0,0.0,41.0,11.0,83.0,14.0,31.0,19.0,...,31.0,9.0,14.0,25.0,91.0,124.0,95.0,3.0,93.0,13.0
122,126,74.0,50.0,0.0,189.0,0.0,142.0,72.0,97.0,50.0,...,160.0,36.0,0.0,28.0,168.0,224.0,118.0,56.0,157.0,31.0


In [65]:
pivoted_df.to_csv('final3.csv')

In [67]:
rawStoreInfDF=pd.merge(rawStoreInfDF,pivoted_df, on='shop_id')

In [68]:
rawStoreInfDF

,shop_id,shop_area_sq_ft,shop_profile,0,1,2,3,4,5,6,...,27,28,29,30,31,32,33,34,35,36
0,47,528,2,22.0,48.0,0.0,144.0,17.0,69.0,44.0,...,65.0,34.0,25.0,20.0,47.0,32.0,59.0,16.0,63.0,15.0
1,9,676,3,63.0,209.0,0.0,394.0,39.0,147.0,115.0,...,182.0,109.0,10.0,67.0,84.0,129.0,79.0,21.0,74.0,34.0
2,83,676,1,44.0,98.0,0.0,338.0,38.0,121.0,109.0,...,205.0,79.0,15.0,31.0,87.0,83.0,80.0,41.0,64.0,36.0
3,117,676,1,55.0,63.0,0.0,270.0,31.0,78.0,166.0,...,177.0,49.0,24.0,27.0,243.0,233.0,245.0,44.0,176.0,23.0
4,42,676,1,51.0,101.0,0.0,286.0,24.0,117.0,81.0,...,148.0,48.0,11.0,30.0,89.0,124.0,88.0,44.0,53.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,87,527,0,19.0,47.0,0.0,201.0,9.0,80.0,53.0,...,109.0,37.0,12.0,18.0,44.0,60.0,72.0,15.0,64.0,17.0
120,50,411,0,39.0,53.0,0.0,501.0,0.0,135.0,116.0,...,315.0,62.0,0.0,37.0,111.0,123.0,141.0,22.0,82.0,21.0
121,61,699,0,46.0,54.0,0.0,312.0,20.0,66.0,108.0,...,164.0,40.0,8.0,15.0,101.0,94.0,148.0,46.0,67.0,37.0
122,56,597,0,82.0,174.0,0.0,604.0,53.0,219.0,185.0,...,298.0,117.0,14.0,58.0,79.0,147.0,93.0,94.0,84.0,39.0


In [75]:
merged_df

,shop_id,num_of_transactions,payment
0,1,1719,1002935
1,2,4115,2645315
2,3,3199,1667950
3,4,3000,1420415
4,5,3421,1610210
...,...,...,...
119,123,1714,825390
120,124,3743,2548215
121,125,1449,966570
122,126,2759,1623175


In [72]:
# convert all float columns to integer columns
rawStoreInfDF = rawStoreInfDF.applymap(lambda x: int(x) if isinstance(x, float) else x)
rawStoreInfDF

,shop_id,shop_area_sq_ft,shop_profile,0,1,2,3,4,5,6,...,27,28,29,30,31,32,33,34,35,36
0,47,528,2,22,48,0,144,17,69,44,...,65,34,25,20,47,32,59,16,63,15
1,9,676,3,63,209,0,394,39,147,115,...,182,109,10,67,84,129,79,21,74,34
2,83,676,1,44,98,0,338,38,121,109,...,205,79,15,31,87,83,80,41,64,36
3,117,676,1,55,63,0,270,31,78,166,...,177,49,24,27,243,233,245,44,176,23
4,42,676,1,51,101,0,286,24,117,81,...,148,48,11,30,89,124,88,44,53,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,87,527,0,19,47,0,201,9,80,53,...,109,37,12,18,44,60,72,15,64,17
120,50,411,0,39,53,0,501,0,135,116,...,315,62,0,37,111,123,141,22,82,21
121,61,699,0,46,54,0,312,20,66,108,...,164,40,8,15,101,94,148,46,67,37
122,56,597,0,82,174,0,604,53,219,185,...,298,117,14,58,79,147,93,94,84,39


In [74]:
rawStoreInfDF=pd.merge(rawStoreInfDF,merged_df, on='shop_id')
rawStoreInfDF

,shop_id,shop_area_sq_ft,shop_profile,0,1,2,3,4,5,6,...,31,32,33,34,35,36,num_of_transactions_x,payment_x,num_of_transactions_y,payment_y
0,47,528,2,22,48,0,144,17,69,44,...,47,32,59,16,63,15,1396,697510,1396,697510
1,9,676,3,63,209,0,394,39,147,115,...,84,129,79,21,74,34,3688,1605555,3688,1605555
2,83,676,1,44,98,0,338,38,121,109,...,87,83,80,41,64,36,2981,1476445,2981,1476445
3,117,676,1,55,63,0,270,31,78,166,...,243,233,245,44,176,23,3544,2033385,3544,2033385
4,42,676,1,51,101,0,286,24,117,81,...,89,124,88,44,53,31,2683,1133820,2683,1133820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,87,527,0,19,47,0,201,9,80,53,...,44,60,72,15,64,17,1713,1048015,1713,1048015
120,50,411,0,39,53,0,501,0,135,116,...,111,123,141,22,82,21,3554,2133300,3554,2133300
121,61,699,0,46,54,0,312,20,66,108,...,101,94,148,46,67,37,2761,1983470,2761,1983470
122,56,597,0,82,174,0,604,53,219,185,...,79,147,93,94,84,39,4846,2494620,4846,2494620


In [81]:
rawStoreInfDF.to_csv('storeinfo.csv')

In [82]:
rawStoreInfDF=pd.read_csv('storeinfo.csv')